# Setup

In [ ]:
import os
from pathlib import Path
import sys

# If we're using Google Colab, we set the environment variable to point to the relevant folder in our Google Drive:
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
    os.environ['SKIN_LESION_CLASSIFICATION'] = '/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification'

# Otherwise, we use the environment variable on our local system:
project_environment_variable = "SKIN_LESION_CLASSIFICATION"

# Path to the root directory of the project:
project_path = Path(os.environ.get(project_environment_variable))

# Relative path to /scripts (from where custom modules will be imported):
scripts_path = project_path.joinpath("scripts")

# Add this path to sys.path so that Python will look there for modules:
sys.path.append(str(scripts_path))

# Now import path_step from our custom utils module to create a dictionary to all subdirectories in our root directory:
from utils import path_setup
path = path_setup.subfolders(project_path)

Mounted at /content/drive
path['project'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification
path['models'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models
path['notebooks'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/notebooks
path['scripts'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/scripts
path['images'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/images


<a id='baseline_models'></a>
# Baseline models

In [ ]:
from typing import Type, Union      # For type hints
from processing import process      # Custom module for processing metadata

data_dir: Path = path["images"]     # Path to directory containing metadata.csv file
csv_filename: str = "metadata.csv"  # The filename

tvr: int = 3              # Ratio of training set to validation set. See discussion below for explanation.
seed: int = 0             # Random seed for parts of the process where randomness is called for.
keep_first: bool = False  # If False, then, for each lesion, we choose a random image to assign to our training set.
stratified: bool = True   # If True, we stratify classes so that the proportions remain as stable as possible after train/val split.
                          # If False, the proportions will be roughly similar.

to_classify: Union[list, dict] = ["mel",   # These are the lesion types we are interested in classifying.
                                  "bcc",   # Any missing ones will be grouped together as the 0-label class: no need to write "other" here.
                                  "akiec", # If 'other' is not desired, use restrict_to attribute above
                                  "nv",]   # Can also be a dictionary, like { 'malignant' : ['mel', 'bcc'], 'benign' : ['nv', 'bkl']}

train_one_img_per_lesion: Union[None, bool] = True

In [ ]:
# Create an instance of the process class with attribute values as above.
rn18_defaults = process(data_dir=data_dir,
                        csv_filename=csv_filename,
                        tvr=tvr,
                        seed=seed,
                        keep_first=keep_first,
                        stratified=stratified,
                        to_classify=to_classify,
                        train_one_img_per_lesion=train_one_img_per_lesion,)

- Loaded file '/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/images/metadata.csv'.
- Inserted 'num_images' column in dataframe, to the right of 'lesion_id' column.
- Inserted 'label' column in dataframe, to the right of 'dx' column: 
  {'df': 0, 'bkl': 0, 'vasc': 0, 'akiec': 1, 'nv': 2, 'bcc': 3, 'mel': 4}
- Added 'set' column to dataframe, with values 't1', 'v1', 'ta', and 'va', to the right of 'localization' column.
- Basic, overall dataframe (pre-train/test split): self.df
- Training set (not balanced, one image per lesion): self.df_train
- Validation set (not expanded, one image per lesion): self.df_val1
- Validation set (not expanded, use all images of each lesion): self.df_val_a
- Small sample dataframes for code testing: self._df_train_code_test, self._df_val1_code_test, self._df_val_a_code_test


In [ ]:
import torchvision.transforms as transforms
transform = transforms.Compose([
transforms.Resize((224,224)), # Resize images to fit ResNet input size
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])

In [ ]:
import pandas as pd
from typing import Union, List, Callable
import torchvision.models as models

source: Union[process, pd.DataFrame] = rn18_defaults  # Processed data to be fed into model for training.
                                                      # Must either be an instance of the process class, or a dataframe of the same format as source.df if source were an instance of the process class.
model_dir: Path = path["models"]                      # Path to directory where models/model info/model results are stored.

transform: Union[None,
                 transforms.Compose,
                 List[Callable]] = transform     # Transform to be applied to images before feeding into neural network.

filename_stem: Union[None, str] = "rn18"         # For saving model and related files. Default "rn18" (if ResNet model) or "EffNet" (if EfficientNet), or "cnn".
filename_suffix: Union[None, str] = "defaults"   # Something descriptive and unique for future reference. Default empty string "".

# model: Union[None, models.ResNet, models.EfficientNet] = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT) # Pre-trained model. Default: ResNet18.
model: Union[None, models.ResNet, models.EfficientNet] = models.resnet18(weights="ResNet18_Weights.DEFAULT")

In [ ]:
# Create an instance of the resnet18 class with attribute values as above.
from multiclass_models import cnn

rn18_defaults = cnn(source=source,
                    model_dir=model_dir,
                    transform=transform,
                    filename_stem=filename_stem,
                    filename_suffix=filename_suffix,
                    model=model,)

New files will be created. 
Base filename: rn18_t1_10e_defaults_00
Attributes saved to file: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_t1_10e_defaults_00_attributes.json


In [ ]:
# from utils import print_header
import torch
import torch.nn as nn

instance = rn18_defaults

model = models.resnet18()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(instance.label_codes))

# state_dict = torch.load(file_path_pth)
# model.load_state_dict(state_dict)

# model = models.efficientnet_b0()
# num_ftrs = model.classifier[1].in_features
# model.classifier[1] = nn.Linear(num_ftrs, len(instance.label_codes))

instance.model = model

file_path = instance.model_dir.joinpath(instance._filename + ".pth")
print(f"Saving model.state_dict() as {file_path}.")
torch.save(model.state_dict(), file_path)

Saving model.state_dict() as /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_t1_10e_defaults_00.pth.


In [ ]:
from utils import print_header
import torch
import torch.nn as nn

instance = rn18_defaults

if instance.state_dict is None:
    print("Loading model and state dictionary from file\n".upper())
    file_path_pth = instance.model_dir.joinpath(instance._filename + ".pth")

    # model = models.efficientnet_b0()
    model = models.resnet18()
    if isinstance(model,models.ResNet):
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, len(instance.label_codes))
    elif isinstance(model,models.EfficientNet):
        num_ftrs = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_ftrs, len(instance.label_codes))

    # Load the state dictionary into the model
    state_dict = torch.load(file_path_pth)
    model.load_state_dict(state_dict)

    instance.model = model
    instance.state_dict = state_dict

print_header("Model architecture")
print(f"Note: \'out_features = {len(instance.label_codes)}\' at the end".upper())
display(instance.model)

LOADING MODEL AND STATE DICTIONARY FROM FILE


MODEL ARCHITECTURE

NOTE: 'OUT_FEATURES = 5' AT THE END


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
from utils import print_header
from multiclass_models import get_probabilities

instance = rn18_defaults

instance.df_probabilities_val1 = get_probabilities(df=instance.df_val1,
                                                   data_dir=instance.data_dir,
                                                   model_dir=instance.model_dir,
                                                   model=instance.model,
                                                   filename=instance._filename,
                                                   label_codes=instance.label_codes,
                                                   transform=instance.transform,
                                                   batch_size=instance.batch_size,
                                                   Print=False,
                                                   save_as=instance._filename + "_val1",)

instance.df_probabilities_val_a = get_probabilities(df=instance.df_val_a,
                                                    data_dir=instance.data_dir,
                                                    model_dir=instance.model_dir,
                                                    model=instance.model,
                                                    filename=instance._filename,
                                                    label_codes=instance.label_codes,
                                                    transform=instance.transform,
                                                    batch_size=instance.batch_size,
                                                    Print=False,
                                                    save_as=instance._filename + "_val_a",)

Saving probabilities: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_t1_10e_defaults_00_val1_probabilities.csv
Saving probabilities: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_t1_10e_defaults_00_val_a_probabilities.csv


In [ ]:
instance = rn18_defaults

file_path1 = instance.model_dir.joinpath("rn18_t1_10e_defaults_00_val1_probabilities.csv")
file_path_a = instance.model_dir.joinpath("rn18_t1_10e_defaults_00_val_a_probabilities.csv")

instance.df_probabilities_val1 = pd.read_csv(file_path1, index_col=0)
instance.df_probabilities_val_a = pd.read_csv(file_path_a, index_col=0)

In [ ]:
instance = rn18_defaults

print_header("Validation set: one image per lesion")
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')]
display(instance.df_probabilities_val1[display_columns].head())

print_header("Validation set: all images per lesion")
display(instance.df_probabilities_val_a[display_columns].head())


VALIDATION SET: ONE IMAGE PER LESION



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_nv,prob_bcc,prob_mel
0,HAM_0002730,ISIC_0025661,bkl,0.113282,0.139375,0.168502,0.489714,0.089128
1,HAM_0001466,ISIC_0027850,bkl,0.086775,0.126703,0.156164,0.560523,0.069836
2,HAM_0002761,ISIC_0029068,bkl,0.121921,0.150040,0.169858,0.454294,0.103887
3,HAM_0004234,ISIC_0029396,bkl,0.132740,0.148602,0.172496,0.433983,0.112180
4,HAM_0001949,ISIC_0025767,bkl,0.113447,0.129855,0.150094,0.511760,0.094844



VALIDATION SET: ALL IMAGES PER LESION



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_nv,prob_bcc,prob_mel
0,HAM_0002730,ISIC_0026769,bkl,0.117399,0.135429,0.161710,0.501293,0.084169
1,HAM_0002730,ISIC_0025661,bkl,0.113282,0.139375,0.168502,0.489714,0.089128
2,HAM_0001466,ISIC_0031633,bkl,0.104984,0.126994,0.166309,0.534228,0.067485
3,HAM_0001466,ISIC_0027850,bkl,0.086775,0.126703,0.156164,0.560523,0.069836
4,HAM_0002761,ISIC_0029176,bkl,0.128270,0.150808,0.166434,0.453231,0.101257


In [ ]:
from collections import OrderedDict
from typing import Dict, List
from multiclass_models import final_prediction

instance = rn18_defaults

raw_probabilities_df1: pd.DataFrame = instance.df_probabilities_val1
raw_probabilities_df_a: pd.DataFrame = instance.df_probabilities_val_a
aggregate_method: Union[None, Dict[str, List[str]]] = { 'max' : ['mel', 'bcc', 'akiec'], 'min' : ['nv'], 'mean' : ['other']}
threshold_dict_help: Union[None, OrderedDict[str, float]] = OrderedDict([('mel',0.4), ('bcc', 0.4), ('akiec', 0.4)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = OrderedDict([('nv',0.6)])
votes_to_win_dict: Union[None, OrderedDict[str, int]] = OrderedDict([('mel',1), ('bcc',1), ('akiec',1)])
label_codes: Dict[int, str] = instance.label_codes
prefix: Union[None, str] = 'prob_'

print_header("Validation set, one image per lesion: combining probabilities and making predictions")

instance.df_pred_val1 = final_prediction(raw_probabilities_df=raw_probabilities_df1,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val1)

print_header("Validation set, all images per lesion: combining probabilities, making predictions, and combining predictions")

instance.df_pred_val_a = final_prediction(raw_probabilities_df=raw_probabilities_df_a,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val_a)


VALIDATION SET, ONE IMAGE PER LESION: COMBINING PROBABILITIES AND MAKING PREDICTIONS



,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_akiec,prob_nv,prob_bcc,prob_mel,pred,pred_final
0,HAM_0002730,2,ISIC_0025661,bkl,0,histo,80.0,male,scalp,v1,0.113282,0.139375,0.168502,0.489714,0.089128,3,3
1,HAM_0001466,2,ISIC_0027850,bkl,0,histo,75.0,male,ear,v1,0.086775,0.126703,0.156164,0.560523,0.069836,3,3
2,HAM_0002761,2,ISIC_0029068,bkl,0,histo,60.0,male,face,v1,0.121921,0.150040,0.169858,0.454294,0.103887,3,3
3,HAM_0004234,2,ISIC_0029396,bkl,0,histo,85.0,female,chest,v1,0.132740,0.148602,0.172496,0.433983,0.112180,3,3
4,HAM_0001949,2,ISIC_0025767,bkl,0,histo,70.0,male,trunk,v1,0.113447,0.129855,0.150094,0.511760,0.094844,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864,HAM_0003949,1,ISIC_0027254,akiec,1,histo,75.0,male,face,v1,0.125690,0.150061,0.175913,0.465067,0.083269,3,3
1865,HAM_0003910,1,ISIC_0026194,akiec,1,histo,30.0,male,face,v1,0.117265,0.131312,0.170949,0.498500,0.081974,3,3
1866,HAM_0001474,1,ISIC_0026765,akiec,1,histo,55.0,female,chest,v1,0.107689,0.129766,0.164313,0.524587,0.073646,3,3
1867,HAM_0001152,1,ISIC_0030133,akiec,1,histo,65.0,male,face,v1,0.142033,0.150675,0.188550,0.424153,0.094588,3,3



VALIDATION SET, ALL IMAGES PER LESION: COMBINING PROBABILITIES, MAKING PREDICTIONS, AND COMBINING PREDICTIONS



,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_akiec,prob_nv,prob_bcc,prob_mel,pred,pred_final
0,HAM_0002730,2,ISIC_0026769,bkl,0,histo,80.0,male,scalp,va,0.117399,0.135429,0.161710,0.501293,0.084169,3,3
1,HAM_0002730,2,ISIC_0025661,bkl,0,histo,80.0,male,scalp,v1,0.113282,0.139375,0.168502,0.489714,0.089128,3,3
2,HAM_0001466,2,ISIC_0031633,bkl,0,histo,75.0,male,ear,va,0.104984,0.126994,0.166309,0.534228,0.067485,3,3
3,HAM_0001466,2,ISIC_0027850,bkl,0,histo,75.0,male,ear,v1,0.086775,0.126703,0.156164,0.560523,0.069836,3,3
4,HAM_0002761,2,ISIC_0029176,bkl,0,histo,60.0,male,face,va,0.128270,0.150808,0.166434,0.453231,0.101257,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2527,HAM_0001474,1,ISIC_0026765,akiec,1,histo,55.0,female,chest,v1,0.107689,0.129766,0.164313,0.524587,0.073646,3,3
2528,HAM_0001152,1,ISIC_0030133,akiec,1,histo,65.0,male,face,v1,0.142033,0.150675,0.188550,0.424153,0.094588,3,3
2529,HAM_0004282,3,ISIC_0033811,akiec,1,histo,65.0,female,face,va,0.150544,0.169365,0.199663,0.359855,0.120574,3,3
2531,HAM_0004282,3,ISIC_0033358,akiec,1,histo,65.0,female,face,va,0.127697,0.157126,0.182498,0.442611,0.090068,3,3


In [ ]:
# from utils import print_header
from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = rn18_defaults
map_labels = instance.label_codes

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']

txp1 = pd.crosstab(target1,prediction1,margins=True,dropna=False)
txp_a = pd.crosstab(target_a,prediction_a,margins=True,dropna=False)

beta = 1
# Weights inversely proportional to relative class size in the training set, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

instance.cm1 = confusion_matrix_with_metric(AxB=txp1,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

instance.cm_a = confusion_matrix_with_metric(AxB=txp_a,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

print_header("Confusion matrix: validation set, one image per lesion")
display(instance.cm1.fillna('_'))

print_header("Confusion matrix: validation set, all images per lesion")
display(instance.cm_a.fillna('_'))


CONFUSION MATRIX: VALIDATION SET, ONE IMAGE PER LESION



predicted,other,akiec,nv,bcc,mel,All,recall
actual,,,,,,,
other,0,0,0,225,0,225,0.0
akiec,0,0,0,57,0,57,0.0
nv,0,0,0,"1,351",0,"1,351",0.0
bcc,0,0,0,82,0,82,1.0
mel,0,0,0,154,0,154,0.0
All,0,0,0,"1,869",0,"1,869",_
precision,_,_,_,0.043874,_,_,_



CONFUSION MATRIX: VALIDATION SET, ALL IMAGES PER LESION



predicted,other,akiec,nv,bcc,mel,All,recall
actual,,,,,,,
other,0,0,0,225,0,225,0.0
akiec,0,0,0,57,0,57,0.0
nv,0,0,0,"1,351",0,"1,351",0.0
bcc,0,0,0,82,0,82,1.0
mel,0,0,0,154,0,154,0.0
All,0,0,0,"1,869",0,"1,869",_
precision,_,_,_,0.043874,_,_,_


In [ ]:
# from utils import print_header
from evaluation import metric_dictionary
# import pandas as pd

instance = rn18_defaults

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']
probabilities1 = instance.df_probabilities_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']
probabilities_a = instance.df_probabilities_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

beta = 1
# Weights inversely proportional to relative class size, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

print_header("Baseline model: other metrics")

instance.metric_dict1 = metric_dictionary(target=target1,
                                          prediction=prediction1,
                                          probabilities=probabilities1)

instance.metric_dict_a = metric_dictionary(target=target_a,
                                          prediction=prediction_a,
                                          probabilities=probabilities_a)

print("\nOne image per lesion".upper())
display(pd.DataFrame(instance.metric_dict1))

print("\nAll images per lesion".upper())
display(pd.DataFrame(instance.metric_dict_a))


BASELINE MODEL: OTHER METRICS


ONE IMAGE PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.043874,0.2,NaN,NaN,0.010849,0.016812,0.037324,0.0,0.51274,0.558064,0.528287



ALL IMAGES PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.043874,0.2,NaN,NaN,0.010849,0.016812,0.037324,0.0,0.514887,0.565971,0.530176
